<a href="https://colab.research.google.com/github/Soronp/cse299/blob/Soron/JohnGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install torch
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.3 MB/s eta 0:00:00


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed

In [8]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [10]:
set_seed(11111)

In [11]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-1b7")
model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-1b7")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

In [162]:
text_prompt = "Explain the reason World War II ended"

In [163]:
prompt_length = len(text_prompt)

In [164]:
length_multiplier = 3

In [165]:
max_length = min(200, prompt_length * length_multiplier)

In [166]:
input_tokens = tokenizer(text_prompt, return_tensors="pt")

In [178]:
with torch.no_grad():
    result_sample = model.generate(
        **input_tokens,
        max_length=max_length,
        do_sample=True,
        top_p=0.85,
        top_k=50,
        temperature=0.7,
        num_return_sequences=1
    )

In [179]:
generated_text = tokenizer.decode(result_sample[0], skip_special_tokens=True)

In [180]:
prompt_embedding = model.get_input_embeddings()(torch.tensor(tokenizer.encode(text_prompt, add_special_tokens=False)).unsqueeze(0)).detach().cpu().numpy()
generated_embedding = model.get_input_embeddings()(torch.tensor(tokenizer.encode(generated_text, add_special_tokens=False)).unsqueeze(0)).detach().cpu().numpy()

In [181]:
max_length = max(prompt_embedding.shape[1], generated_embedding.shape[1])

In [182]:
prompt_pad_length = max_length - prompt_embedding.shape[1]
generated_pad_length = max_length - generated_embedding.shape[1]

In [183]:
prompt_embedding = np.pad(prompt_embedding, ((0, 0), (0, max_length - prompt_embedding.shape[1]), (0, 0)), mode='constant', constant_values=0)
generated_embedding = np.pad(generated_embedding, ((0, 0), (0, max_length - generated_embedding.shape[1]), (0, 0)), mode='constant', constant_values=0)

In [184]:
prompt_embedding = prompt_embedding.squeeze(axis=0)
generated_embedding = generated_embedding.squeeze(axis=0)

In [185]:
similarity_scores = []
for i in range(min(prompt_embedding.shape[1], generated_embedding.shape[1])):
    similarity_score = cosine_similarity(prompt_embedding[:, i].reshape(1, -1),
                                          generated_embedding[:, i].reshape(1, -1))
    similarity_scores.append(similarity_score)

In [186]:
accuracy_percentage = (1 - np.abs(1 - similarity_score)) * 100

In [187]:
print("Generated Text:\n", generated_text)

Generated Text:
 Explain the reason World War II ended, the Japanese surrender, and the end of the Cold War.


In [188]:
print("Accuracy Percentage:", accuracy_percentage)

Accuracy Percentage: [[74.55233]]
